In [1]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O horses_and_humans.zip

--2020-07-02 10:13:04--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 172.217.214.128, 108.177.111.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horses_and_humans.zip’

horses_and_humans.z 100%[===================>] 142.65M   127MB/s    in 1.1s    

2020-07-02 10:13:06 (127 MB/s) - ‘horses_and_humans.zip’ saved [149574867/149574867]



In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile

In [3]:
class myCallbacks(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if (logs.get('accuracy')>0.75):
      print('\n Reached 75% accuracy on Training Set')
      self.model.stop_training = True

In [4]:
callbacks = myCallbacks()

In [5]:
os.listdir()

['.config', 'horses_and_humans.zip', 'sample_data']

In [6]:
myzip = 'horses_and_humans.zip'
zip_use = zipfile.ZipFile(myzip,'r')
zip_use.extractall('horses_and_humans')
zip_use.close()

In [7]:
os.listdir()

['.config', 'horses_and_humans', 'horses_and_humans.zip', 'sample_data']

In [8]:
os.listdir('horses_and_humans')

['horses', 'humans']

In [9]:
horses_dir = 'horses_and_humans/horses'
humans_dir = 'horses_and_humans/humans'

In [10]:
horse_names = list(os.listdir(horses_dir))
human_names = list(os.listdir(humans_dir))

In [11]:
print(horse_names[:10],human_names[:10])

['horse43-5.png', 'horse16-1.png', 'horse50-7.png', 'horse38-2.png', 'horse27-1.png', 'horse15-9.png', 'horse37-1.png', 'horse05-8.png', 'horse28-5.png', 'horse41-0.png'] ['human15-17.png', 'human12-10.png', 'human12-11.png', 'human17-15.png', 'human01-23.png', 'human10-16.png', 'human10-08.png', 'human04-28.png', 'human10-28.png', 'human06-03.png']


In [12]:
print(len(horse_names),len(human_names))

500 527


In [13]:
#follow the following code to view the images , this step can be skiped 
import matplotlib.image as mpimg
nrows = 4
ncols = 4
pic_index = 0
#will be completed/updated later on . Right now focusing on the model .

In [14]:
model = tf.keras.Sequential([
                             tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(300,300,3)),
                             tf.keras.layers.MaxPooling2D(2,2),
                             tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
                             tf.keras.layers.MaxPooling2D(2,2),
                             tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                             tf.keras.layers.MaxPooling2D(2,2),
                             tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                             tf.keras.layers.MaxPooling2D(2,2),
                             tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                             tf.keras.layers.MaxPooling2D(2,2),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(512,activation='relu'),
                             tf.keras.layers.Dense(1,activation='sigmoid')
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [16]:
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.RMSprop(lr=0.001),metrics=['accuracy'])

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator as IDG

In [18]:
train_datagen = IDG(rescale=1/255.0,rotation_range=0.2,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.2,zoom_range=0.2,
                    horizontal_flip=True,fill_mode='nearest')

In [19]:
train_generator = train_datagen.flow_from_directory('horses_and_humans',target_size=(300,300),batch_size=128,class_mode='binary')

Found 1027 images belonging to 2 classes.


In [20]:
model.fit_generator(train_generator,steps_per_epoch=8,epochs=15,callbacks=[callbacks])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
8/8 [==============================] - 21s 3s/step - loss: 0.7508 - accuracy: 0.5352
Epoch 2/15
8/8 [==============================] - 20s 2s/step - loss: 0.6860 - accuracy: 0.5406
Epoch 3/15
8/8 [==============================] - 20s 3s/step - loss: 0.6813 - accuracy: 0.6118
Epoch 4/15
8/8 [==============================] - 23s 3s/step - loss: 0.6543 - accuracy: 0.6807
Epoch 5/15
8/8 [==============================] - 20s 3s/step - loss: 0.5680 - accuracy: 0.7408
Epoch 6/15
8/8 [==============================] - ETA: 0s - loss: 0.4990 - accuracy: 0.7519
 Reached 75% accuracy on Training Set
8/8 [==============================] - 20s 3s/step - loss: 0.4990 - accuracy: 0.7519


In [21]:
from google.colab import files
from tensorflow.keras.preprocessing import image
uploaded = files.upload()
for fn in uploaded.keys():
  path = '/content/' + fn
  img = image.load_img(path,target_size=(300,300))
  x = image.img_to_array(img)
  x = np.expand_dims(x,axis=0)
  images = np.vstack([x])
  classes = model.predict(images,batch_size=10)
  print(classes)
  if classes[0] > 0.5:
    print(fn+' is a Human')
  else :
    print(fn+' is a Horse')  

Saving humanimage5.jpg to humanimage5.jpg
Saving humanimage4.jpg to humanimage4.jpg
Saving humanwomenimages3.jpg to humanwomenimages3.jpg
Saving humanimage2.jpg to humanimage2.jpg
Saving humanimage1.jpg to humanimage1.jpg
Saving horseimage3.jpg to horseimage3.jpg
Saving horseimage2.jpg to horseimage2.jpg
Saving horseimage1.jpg to horseimage1.jpg
[[1.]]
humanimage5.jpg is a Human
[[1.]]
humanimage4.jpg is a Human
[[0.]]
humanwomenimages3.jpg is a Horse
[[1.]]
humanimage2.jpg is a Human
[[1.]]
humanimage1.jpg is a Human
[[0.]]
horseimage3.jpg is a Horse
[[1.]]
horseimage2.jpg is a Human
[[2.4263841e-15]]
horseimage1.jpg is a Horse
